In [1]:
import sys
import time
import random

import numpy as np
import gym
import gym_chess
import pandas as pd
import numpy as np
import torch
from torch import nn

In [ ]:
#le NN prend comme entrée l'état et en sortie un array de 64 * 64 + 4 elements qui correspond aux 64 * 64 + 4 coups possibles

# Taille des entrées et sorties
input_size = 8 * 8  # État de l'échiquier aplati
output_size = 64 * 64 + 4  # 4100 coups possibles

# Modèle du réseau
model = nn.Sequential(
    nn.Flatten(),                     # Aplatit l'entrée (8x8 -> 64)
    nn.Linear(input_size, 256),       # Première couche cachée avec 256 neurones
    nn.ReLU(),                        # Fonction d'activation ReLU
    nn.Linear(256, 512),              # Deuxième couche cachée avec 512 neurones
    nn.ReLU(),                        # Fonction d'activation ReLU
    nn.Linear(512, 1024),             # Troisième couche cachée avec 1024 neurones
    nn.ReLU(),                        # Fonction d'activation ReLU
    nn.Linear(1024, 512),             # Quatrième couche cachée avec 512 neurones
    nn.ReLU(),                        # Fonction d'activation ReLU
    nn.Linear(512, 256),              # Cinquième couche cachée avec 256 neurones
    nn.ReLU(),                        # Fonction d'activation ReLU
    nn.Linear(256, output_size),      # Couche de sortie avec 4100 neurones
    nn.Softmax(dim=-1)                # Fonction Softmax pour des probabilités sur les coups possibles
)

# Afficher le modèle
print(model)

In [ ]:
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(lr=0.001, params = model.parameters())

In [3]:
df = pd.read_csv("games.csv")

In [5]:
display(df.iloc[0]["moves"])

'd4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5 Bf4'

In [76]:

env = gym.make("ChessVsSelf-v1", log=False)

#
# Play against self
#
num_episodes = 2
num_steps = 100

total_steps = 0
collected_rewards = []
start = time.time()

[[-3 -5 -4 -2 -1 -4 -5 -3]
 [-6 -6 -6 -6 -6 -6 -6 -6]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 6  6  6  6  6  6  6  6]
 [ 3  5  4  2  1  4  5  3]]


In [ ]:
def list_move_to_string(moves:list):
  
  for idx, move in enumerate(moves):
                  
    moves[idx] = env.move_to_string(move)
  
  return moves
      

In [77]:


for i in range(num_episodes):
    initial_state = env.reset()
    print("\n", "=" * 10, "NEW GAME", "=" * 10)
    env.render()
    total_rewards = {"WHITE": 0, "BLACK": 0}
    for game in df:
        #On récupère les coups
        chess_moves = game["moves"]
        
        #on transforme le str en une liste de coups
        moves_list = chess_moves.split()
        i = 0
        #on iter sur les coups sous pgn notation
        for move_pgn in moves_list:
            #on récupère l'état
            state = env.state
            
            #On réalise la prédiction
            output_vector = model(state)
            
            #on transforme output_vector en action
            output_action = torch.argmax(output_vector).item()
            
            #On récupère la liste des coups possibles
            possible_moves = env.possible_moves
            
            #On transforme les coups en pgn
            possible_moves_pgn = list_move_to_string(possible_moves)
            
            #On récupère l'indice de move_pgn dans possible_moves_pgn
            index = possible_moves_pgn.index(move_pgn)
            
            #On récupère le bon coup
            m = possible_moves[index]
            
            #On le transforme en action
            a = env.move_to_action(m)
            
            action_one_hot = torch.zeros(64 * 64 + 4)  # Crée un tenseur rempli de 0
            action_one_hot[a] = 1  
            
            #On calcul la loss
            loss = loss_function(output_vector, action_one_hot)
            
            #On met à jour le réseau
            optimizer.zero_grad() # Mettre les gradients à 0
            loss.backward()
            optimizer.step()
            
            # perform action
            state, reward, done, _ = env.step(a)
            i += 1
            
            if done or i > num_steps:
                break
                
                
                
                
                
                
            

[[array([6, 0], dtype=int8), array([5, 0], dtype=int8)], [array([6, 0], dtype=int8), array([4, 0], dtype=int8)], [array([6, 1], dtype=int8), array([5, 1], dtype=int8)], [array([6, 1], dtype=int8), array([4, 1], dtype=int8)], [array([6, 2], dtype=int8), array([5, 2], dtype=int8)], [array([6, 2], dtype=int8), array([4, 2], dtype=int8)], [array([6, 3], dtype=int8), array([5, 3], dtype=int8)], [array([6, 3], dtype=int8), array([4, 3], dtype=int8)], [array([6, 4], dtype=int8), array([5, 4], dtype=int8)], [array([6, 4], dtype=int8), array([4, 4], dtype=int8)], [array([6, 5], dtype=int8), array([5, 5], dtype=int8)], [array([6, 5], dtype=int8), array([4, 5], dtype=int8)], [array([6, 6], dtype=int8), array([5, 6], dtype=int8)], [array([6, 6], dtype=int8), array([4, 6], dtype=int8)], [array([6, 7], dtype=int8), array([5, 7], dtype=int8)], [array([6, 7], dtype=int8), array([4, 7], dtype=int8)], [array([7, 1], dtype=int8), array([5, 0], dtype=int8)], [array([7, 1], dtype=int8), array([5, 2], dtype

In [78]:
total_steps += 1
            # white moves
            moves = env.possible_moves
            m = random.choice(moves)
            a = env.move_to_action(m)
            # perform action
            state, reward, done, _ = env.step(a)
            total_rewards["WHITE"] += reward
            if done:
                break
    
            # black moves
            moves = env.possible_moves
            m = random.choice(moves)
            a = env.move_to_action(m)
            # perform action
            state, reward, done, _ = env.step(a)
            total_rewards["BLACK"] += reward
            if done:
                break

    print(">" * 5, "GAME", i, "REWARD:", total_rewards)
    collected_rewards.append(total_rewards)

Action: 3364 is being passed to the environment.
           >>>>>>>>>> WHITE
    -------------------------
 8 |  ♖  ♘  ♗  ♕  ♔  ♗  ♘  ♖ |
 7 |  ♙  ♙  ♙  ♙  ♙  ♙  ♙  ♙ |
 6 |  .  .  .  .  .  .  .  . |
 5 |  .  .  .  .  .  .  .  . |
 4 |  .  .  .  .  .  .  .  . |
 3 |  .  .  .  .  .  .  .  . |
 2 |  ♟  ♟  ♟  ♟  ♟  ♟  ♟  ♟ |
 1 |  ♜  ♞  ♝  ♛  ♚  ♝  ♞  ♜ |
    -------------------------
      a  b  c  d  e  f  g  h 
           >>>>>>>>>> BLACK
    -------------------------
 8 |  ♖  ♘  ♗  ♕  ♔  ♗  ♘  ♖ |
 7 |  ♙  ♙  ♙  ♙  ♙  ♙  ♙  ♙ |
 6 |  .  .  .  .  .  .  .  . |
 5 |  .  .  .  .  .  .  .  . |
 4 |  .  .  .  .  ♟  .  .  . |
 3 |  .  .  .  .  .  .  .  . |
 2 |  ♟  ♟  ♟  ♟  .  ♟  ♟  ♟ |
 1 |  ♜  ♞  ♝  ♛  ♚  ♝  ♞  ♜ |
    -------------------------
      a  b  c  d  e  f  g  h 
New State: [[-3 -5 -4 -2 -1 -4 -5 -3]
 [-6 -6 -6  0 -6 -6 -6 -6]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0 -6  0  0  0  0]
 [ 0  0  0  0  6  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 6  6  6  6  0  6  6  6]
 [ 3  5  4  2  1  4